### Imports


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier


### Load Data

In [ ]:
data = pd.read_csv("data/mushrooms.csv")


### Data Exploration

In [ ]:
data.head(10)


Display the distribution of edible and poisonous mushrooms.

In [ ]:
labels = "Edible", "Poisonous"
sizes = [
    data.describe()["class"]["freq"],
    data.describe()["class"]["count"] - data.describe()["class"]["freq"],
]

fig, ax = plt.subplots()
ax.pie(
    sizes,
    labels=labels,
    autopct="%1.1f%%",
    shadow=True,
    startangle=90,
    colors=["green", "magenta"],
)

ax.set_title("Mushroom Edibility")
ax.legend(labels, loc="upper right")
ax.axis("equal")
plt.tight_layout()

plt.show()


Mushroom edibility by cap shape

In [ ]:
cap_shapes = data.groupby(["cap-shape", "class"]).size().unstack()
cap_shapes.fillna(0)
cap_shapes_mapping = {
    "b": "bell",
    "c": "conical",
    "x": "convex",
    "f": "flat",
    "k": "knobbed",
    "s": "sunken",
}

cap_shapes.rename(index=cap_shapes_mapping, inplace=True)

ax = cap_shapes.plot.bar(color=["green", "magenta"], rot=0, figsize=(12, 7), width=0.8)

ax.set_title("Mushroom eedibility by cap shape")
ax.set_ylabel("Count")
ax.set_xlabel("Cap Shape")
labels = ["Edible", "Poisonous"]

ax.legend(labels, loc="upper left")
plt.show()


Mushroom edibility by cap surface

In [ ]:
cap_surfaces = data.groupby(["cap-surface", "class"]).size().unstack()
cap_surfaces.fillna(0)

cap_surfaces_mapping = {
    "f": "fibrous",
    "g": "grooves",
    "s": "smooth",
    "y": "scaly",
}

cap_surfaces.rename(index=cap_surfaces_mapping, inplace=True)

ax = cap_surfaces.plot.bar(color=["green", "magenta"], rot=0, figsize=(12, 7), width=0.8)

ax.set_title("Mushroom edibility by cap surface")
ax.set_ylabel("Count")
ax.set_xlabel("Cap Surface")
labels = ["Edible", "Poisonous"]

ax.legend(labels, loc="upper left")
plt.show()


Mushroom edibility by color

In [ ]:
cap_colors = data.groupby(["cap-color", "class"]).size().unstack()
cap_colors.fillna(0)

cap_colors_mapping = {
    "n": "brown",
    "b": "buff",
    "c": "cinnamon",
    "g": "gray",
    "r": "green",
    "p": "pink",
    "u": "purple",
    "e": "red",
    "w": "white",
    "y": "yellow",
}

cap_colors.rename(index=cap_colors_mapping, inplace=True)

ax = cap_colors.plot.bar(color=["green", "magenta"], rot=0, figsize=(12, 7), width=0.8)

ax.set_title("Mushroom edibility by cap color")
ax.set_xlabel("Cap Color")
ax.set_ylabel("Count")

labels = ["Edible", "Poisonous"]
ax.legend(labels, loc="upper left")
plt.show()


Mushroom edibility by bruises

In [ ]:
bruises = data.groupby(["bruises", "class"]).size().unstack()

bruise_mapping = {"f": "no bruises", "t": "bruises"}

bruises.rename(index=bruise_mapping, inplace=True)

ax = bruises.plot.bar(color=["green", "magenta"], rot=0, figsize=(12, 7), width=0.8)

ax.set_title("Mushroom edibility by present bruises")
ax.set_ylabel("Count")

labels = ["Edible", "Poisonous"]
plt.legend(labels, loc="upper left")
plt.show()


### Declare utility functions to imitiate missing data

In [ ]:
def create_missing_data(data, missing_ratio):
    """
    Create random missing data in a pandas DataFrame.

    Parameters:
        data (pandas.DataFrame): The input DataFrame.
        missing_ratio (float): The ratio of missing values to create (between 0 and 1).

    Returns:
        pandas.DataFrame: The DataFrame with random missing values.
    """

    data_with_missing = data.copy()
    num_missing_values = int(data.size * missing_ratio)
    missing_indices = np.random.choice(data.size, size=num_missing_values, replace=False)
    data_with_missing.values.flat[missing_indices] = np.nan

    return data_with_missing


In [ ]:
def fill_with_most_common(col):
    col.fillna(col.value_counts().index[0], inplace=True)
    return col


### Train Models

Split the data into train and test 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop("class", axis=1),
    data["class"],
    test_size=0.2,
    shuffle=True,
    random_state=42,
    stratify=data["class"],
)


## Create missing data

In [ ]:
X_train = create_missing_data(X_train, 0.7)
X_train = X_train.apply(lambda col: fill_with_most_common(col))

X_test = create_missing_data(X_test, 0.7)
X_test = X_test.apply(lambda col: fill_with_most_common(col))


Replace edible with 1 and poisonous with 0

In [ ]:
y_train = y_train.map({"e": 0, "p": 1})
y_test = y_test.map({"e": 0, "p": 1})


### Preprocessing

Create preprocessor

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        (
            "num",
            StandardScaler(),
            data.drop("class", axis=1).select_dtypes(exclude="object").columns,
        ),
        (
            "cat",
            OneHotEncoder(handle_unknown="ignore"),
            data.drop("class", axis=1).select_dtypes(include="object").columns,
        ),
    ]
)


### Create XGBoost model

In [ ]:
xgb_model = XGBClassifier()

xgb_pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", xgb_model)])


Search for the parameters with Grid Search

In [ ]:
param_grid = {
    "classifier__n_estimators": [50, 100, 150],
    "classifier__learning_rate": [0.1, 0.01, 0.001],
}

grid_search = GridSearchCV(xgb_pipeline, param_grid, cv=5, n_jobs=-1, verbose=1)


In [ ]:
grid_search.fit(X_train, y_train)

print("Best parameters set found on development set:")
print(grid_search.best_params_)


Predict scores

In [ ]:
y_pred = grid_search.predict(X_test)

print("Accuracy score: ", accuracy_score(y_test, y_pred))


### Create Neural Network model

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, CategoryEncoding

from sklearn.preprocessing import OneHotEncoder


Split test data into validation and test

In [ ]:
X_validation, X_test, y_validation, y_test = train_test_split(
    X_test, y_test, test_size=0.5, shuffle=True, random_state=42, stratify=y_test
)


Encode input data with OneHotEncoder

In [ ]:
onehot_encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

X_train_onehot = onehot_encoder.fit_transform(X_train)
X_test_onehot = onehot_encoder.transform(X_test)
X_validation_onehot = onehot_encoder.transform(X_validation)

print(X_train_onehot.shape[1])
print(X_test_onehot.shape[1])
print(X_validation_onehot.shape[1])


Create the layers for the neural network

In [ ]:
neural_network = Sequential()

neural_network.add(Dense(units=16, activation="elu", input_shape=(X_train_onehot.shape[1],)))
neural_network.add(Dense(units=16, activation="elu"))
neural_network.add(Dense(units=1, activation="sigmoid"))

neural_network.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [ ]:
nn_history = neural_network.fit(
    X_train_onehot,
    y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_validation_onehot, y_validation),
)


Evaluate neural network

In [ ]:
neural_network.evaluate(X_test_onehot, y_test)


Graph the loss and accuracy

In [ ]:
pd.DataFrame(nn_history.history).plot(figsize=(12, 7))
plt.grid(True)
plt.xlabel("Epoch")
plt.show()


## Convolutional Neural Network

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import (
    Dense,
    CategoryEncoding,
    Conv1D,
    MaxPooling1D,
    Flatten,
    Dropout,
    BatchNormalization,
    Activation,
    Input,
)

from sklearn.preprocessing import OneHotEncoder


Encode the data with OneHotEncoder

In [ ]:
onehot_encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

X_train_onehot = onehot_encoder.fit_transform(X_train)
X_test_onehot = onehot_encoder.transform(X_test)
X_validation_onehot = onehot_encoder.transform(X_validation)

print(X_train_onehot.shape[1])
print(X_test_onehot.shape[1])
print(X_validation_onehot.shape[1])


## Create the layers for CNN

Conv1D Layer (filters=16, kernel_size=3, strides=1, padding="same"):

    This is a 1D convolutional layer that performs convolution on the input data.
    It uses 16 filters, each of size 3, to extract 16 different features from the input.
    The stride of 1 means the filters move one step at a time.
    Padding="same" ensures that the output has the same spatial dimensions as the input.

BatchNormalization:

    This layer normalizes the activations of the previous layer.
    It helps in stabilizing and accelerating the training process by reducing the internal covariate shift.

Activation("relu"):

    ReLU (Rectified Linear Unit) is an activation function that introduces non-linearity.
    It applies the element-wise rectified linear activation to the output of the previous layer.

MaxPooling1D Layer (pool_size=2, strides=2):

    This layer performs max pooling, which downsamples the input by taking the maximum value in each region.
    It uses a pool size of 2, meaning it takes the maximum value within a window of size 2.
    The stride of 2 means the pooling window moves two steps at a time.

Conv1D Layer (filters=32, kernel_size=3, strides=1, padding="same"):

    This is another 1D convolutional layer with 32 filters and a kernel size of 3.
    It extracts 32 different features from the input using convolution.

BatchNormalization:

    Again, this layer normalizes the activations of the previous layer.

Activation("relu"):

    Applies ReLU activation to the output of the previous layer.

MaxPooling1D Layer (pool_size=2, strides=2):

    Performs max pooling with a pool size of 2 and stride of 2.

Flatten:

    This layer flattens the previous layer's output into a 1D vector.
    It converts the multi-dimensional feature maps into a single continuous vector.

Dense Layer (units=16, activation="elu"):

    Adds a fully connected layer with 16 units (neurons).
    It connects every neuron in this layer to every neuron in the previous layer.
    The activation function used here is ELU (Exponential Linear Unit).

Dense Layer (units=16, activation="elu"):

    Adds another fully connected layer with 16 units and ELU activation.

Dense Layer (units=1, activation="sigmoid"):

    Adds a final fully connected layer with 1 unit and sigmoid activation.
    Sigmoid activation is commonly used for binary classification problems to produce a probability output.

In [ ]:
cnn = Sequential()

# create convolutional neural network

cnn.add(
    Conv1D(
        filters=16,
        kernel_size=3,
        strides=1,
        padding="same",
        input_shape=(X_train_onehot.shape[1], 1),
    )
)
cnn.add(BatchNormalization())
cnn.add(Activation("relu"))
cnn.add(MaxPooling1D(pool_size=2, strides=2))

cnn.add(
    Conv1D(
        filters=32,
        kernel_size=3,
        strides=1,
        padding="same",
    )
)

cnn.add(BatchNormalization())
cnn.add(Activation("relu"))
cnn.add(MaxPooling1D(pool_size=2, strides=2))
cnn.add(Flatten())

cnn.add(Dense(units=16, activation="elu"))
cnn.add(Dense(units=16, activation="elu"))
cnn.add(Dense(units=1, activation="sigmoid"))

cnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In the provided code, the input shape of the CNN is defined as (X_train_onehot.shape[1], 1), which means the network expects input data with shape (num_features, 1). However, the one-hot encoded data (X_train_onehot and X_validation_onehot) have a shape of (num_samples, num_features).

To match the expected input shape, the reshape operation is applied to the input data.

This reshapes the input data to have dimensions (num_samples, num_features, 1). The additional dimension of size 1 represents the channel dimension, which is required for convolutional layers.

In [ ]:
cnn_history = cnn.fit(
    X_train_onehot.reshape(X_train_onehot.shape[0], X_train_onehot.shape[1], 1),
    y_train,
    epochs=100,
    batch_size=32,
    validation_data=(
        X_validation_onehot.reshape(X_validation_onehot.shape[0], X_validation_onehot.shape[1], 1),
        y_validation,
    ),
)


Evaluate CNN

In [ ]:
cnn.evaluate(X_test_onehot.reshape(X_test_onehot.shape[0], X_test_onehot.shape[1], 1), y_test)


Plot the loss and accuracy for CNN

In [ ]:
pd.DataFrame(cnn_history.history).plot(figsize=(12, 7))
plt.grid(True)
plt.xlabel("Epoch")
plt.show()
